# InViz: Live Computation Example

In this notebook, we'll see how you can plug any function you want into InViz to help visualize the relationship between parameters and observables.

In [16]:
import inviz as nv
import numpy as np
import pandas as pd
from scipy import signal

Suppose we want to visualize how changes in amplitude, frequency, and phase affect some wave functions. Let's choose the sine and sinc functions, and we'll also throw in a sawtooth wave just for fun.

A sine wave can be defined as:
$$
y(x) = A \sin{(2\pi f x + \phi)} = A \sin{(\omega x + \phi)}
$$
where $A$ is the amplitude, $f$ is the frequency, $\omega$ is the angular frequency, and $\phi$ is the phase.

The sinc function is simply:
$$
\mathrm{sinc}~(x) = \frac{\sin{(x)}}{x}
$$

The sawtooth wave is a bit more complicated. You can read more about it [here](https://en.wikipedia.org/wiki/Sawtooth_wave).

First let's generate some random data:

In [5]:
# Set up the parameters of the problem.
ndim, nsamples = 3, 1000

# Generate some fake data.
np.random.seed(42)
data1 = np.random.randn(ndim * 4 * nsamples // 5).reshape(
    [4 * nsamples // 5, ndim]
)
data2 = 4 * np.random.rand(ndim)[None, :] + np.random.randn(
    ndim * nsamples // 5
).reshape([nsamples // 5, ndim])
data = np.vstack([data1, data2])

Now let's convert it into a pandas DataFrame (InViz only accepts DataFrames, for now) and add labels with some fancy $\LaTeX$ formatting:

In [6]:
param_names = ['frequency', 'phase', 'amplitude']
latex = ['\omega / 2\pi', '\phi', '\mathrm{amplitude}']
df = pd.DataFrame(data, columns=param_names)
latex_dict = dict(zip(param_names, latex))

Next, we will define the function that takes a row of input data and uses it to compute the three waveforms. Note that the function must return data as a list of dicts in the following format:

```python
results = [
    {'x': x, 'a': a},
    {'y': y, 'b': b},
    {'z': z, 'c': c}, 
]
```

The values x, y, z will go on the horizontal axes of each graph, and a, b, c will go on the vertical axes. The keys will be used as the title and axis labels of each respective plot. 

Like so:

In [7]:
def compute_waveforms(index, input_data):
    selection = input_data.iloc[[index]]
    x = np.linspace(-4*np.pi, 4*np.pi, 1000)
    angular_freq = 2*np.pi*selection['frequency'].iloc[0]
    phase = selection['phase'].iloc[0]
    amp = selection['amplitude'].iloc[0]
    sin = amp * np.sin(angular_freq*x + phase)
    cos = amp * np.cos(angular_freq*x + phase)
    sinc = amp * np.sinc(angular_freq*x/np.pi + phase)
    sawtooth = amp * signal.sawtooth(angular_freq * x + phase)
    waves = [
        {'x': x, 'sin(x)': sin},
        {'x': x, 'sinc(x)': sinc},
        {'x': x, 'sawtooth': sawtooth},
    ]
    return waves

Now we are ready to set up an InViz Observable object. This is a way to label your data so that InViz can display it. Observables are flexible, so you can specify a variety of data visualization types. Currently supported are:

- `'Curve'`
- `'Scatter'`
- `'Bars'`

**IMPORTANT**: only pass arguments into __either__ `data_observable` __or__ `myfunction + myfunction_args`, depending on if you have a precalculated observable dataset, or you want to dynamically calculate them with a custom function.

You can also customize your plots using HoloViews Options objects (documentation [here](https://holoviews.org/user_guide/Applying_Customizations.html)). 

In [19]:
import holoviews as hv
from holoviews import opts

opts1 = opts.Curve(xlim=(-4*np.pi, 4*np.pi), height=400, width=500, fontscale=1.1, color=hv.Cycle('YlOrRd'), bgcolor='#151515', framewise=True)
opts2 = opts.Curve(xlim=(-4*np.pi, 4*np.pi), height=400, width=500, fontscale=1.1, color=hv.Cycle('PuBuGn'), bgcolor='#151515', framewise=True)
opts3 = opts.Curve(xlim=(-4*np.pi, 4*np.pi), height=400, width=500, fontscale=1.1, color=hv.Cycle('RdPu'), bgcolor='#f5f5f5', framewise=True)

waves_latex = {
    'x': 'x', 
    'sin(x)': '\sin{x}',
    'sinc(x)': '1/\sin{x}',
    'sawtooth': '\mathrm{Sawtooth~Wave}',
}

waveforms = nv.Observable(
    name=[
        'Sine',
        'Sinc',
        'Sawtooth'
    ],
    myfunc=compute_waveforms,
    myfunc_args=(df,),
    grouped=True,
    plot_type=[
        'Curve',
        'Curve',
        'Curve',
    ],
    plot_opts=[
        opts1,
        opts2,
        opts3,
    ],
    latex_labels=waves_latex
)

Finally, we will use `nv.viz` to interactively visualize the whole thing. Here's an overview of the function arguments:
- __data__: (Pandas DataFrame) the data you want shown as a scatter plot
- __observables__: (list) a list of observables you'd like to visualize
- __show_observables__: (Boolean) whether you want to see the observable plots or not (default = True)
- __latex_dict__: (dict) a dictionary containing the Latex formatting for your axis labels (default = None)

Run the cell below to test out the interactivity by selecting points on the scatterplot in the top section, and see what appears on the plots in the bottom section!

In [20]:
nv.viz(data=df, observables=[waveforms], latex_dict=latex_dict)

Column
    [0] Column
        [0] Row
            [0] Select(name='Horizontal Axis', options=['frequency', ...], value='phase')
            [1] Select(name='Vertical Axis', options=['frequency', ...], value='amplitude')
            [2] Select(name='Colormapped Parameter', options=['frequency', ...], value='frequency')
            [3] Checkbox(align='end', name='Show Colormap', value=True)
        [1] Row
            [0] HoloViews(DynamicMap, height=400, sizing_mode='fixed', width=500)
            [1] ParamFunction(function, _pane=HoloViews)
    [1] HoloViews(DynamicMap)